<h1 style="color:#00A6D6;">Introduction to Quantum Cryptography - Jupyter Notebooks</h1>
<h2 style="color:#00A6D6;">Chapter 7: Distributing keys</h2>

Welcome to the new Julia sheet! As usual, we will ask you to use Julia to answer a few exercises. Most importantly, however, the purpose of these Julia sheets is for you to play around and build intuition by exploring and calculating things we do NOT ask you :-) We hope that you take advantage of using Julia this way.

In this Julia Jupyter notebook, we focus on getting more closely acquainted with doing error-correction!

* <a href="#Enc">Encoder</a>
* <a href="#Dec">Decoder</a>
* <a href="#All">Putting all together</a>

The include command that follows will include all the functions that you have used on the previous weeks including D for trace distance and minEntropy for computing the min-entropy. Please use them as you see fit to answer the exercises.

In [1]:
include("source/main.jl");

   Resolving package versions...
  No Changes to `C:\Users\swehner\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\swehner\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\swehner\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\swehner\.julia\environments\v1.9\Manifest.toml`


WebIO._IJuliaInit()

In this sheet we will implement a toy information reconciliation from scratch. It will be very similar to the scheme described in the book except that we will use different codes. Let us first recall the communication scenario of a one-way information reconciliation protocol: 

<img src="source/oneway.png">

Alice holds a string $X_A$, Bob holds a string $X_B=X_A+S$ and we call $S$ the error vector. In order to help Bob recover $X_A$, Alice will encode her string in $C_A=\textrm{Enc}(X_A)$ and send $C_A$ to Bob via a public authentic channel. Bob will input $C_A$ and $X_B$ into a decoder that will produce an estimate of Alice's string $\hat X_A=\textrm{Dec}(C_A,X_B)$.

<a id=Enc></a>
<h2 style="color:#00A6D6;">Encoder</h2>

Let us first take a look at the encoder function. Given some parity check matrix $H$, the encoder function that we will use will encode $X_A$ into the syndrome of $X_A.$ That is: $C_A=H\cdot X_A^T$. 

So, let us write the encoding function which is simply a small function for computing syndromes:

In [28]:
function getSyndrome(H,v)
    # Input:
    # ------
    # H - k x n parity check matrix
    # v - length n vector as a column 
    #
    # Output:
    # -------
    # syn syndrome of v with parity check matrix H as row vector
    syn = (H * v) .% 2
    return syn
end

getSyndrome (generic function with 1 method)

<h3 style="color:#00A6D6;"> Exercise 1</h3>

Consider the parity check matrices corresponding to some well known codes (Golay and a Hamming Code):

In [3]:
H_Golay = [1 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0;
 1 0 1 0 1 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0;
 1 0 1 1 0 1 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0;
 1 0 1 1 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0;
 1 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0;
 1 1 0 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0;
 1 1 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0;
 1 1 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0;
 1 1 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0;
 1 1 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0;
 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1];

H_Hamming = [1 0 1 0 1 0 1;
             0 1 1 0 0 1 1;
             0 0 0 1 1 1 1];

Let us investigate the codes induced by these matrices:

* How many different syndromes has the Hamming code?
* How many different syndromes has the Golay code?

Recall that the weight of a vector is the number of non-zero entries in the vector. Let $V_n^w$ be the set of binary vectors of length $n$ and weight $w$:

* What is the number of elements in $V_7^0$,$V_7^1$,$V_7^2$ and $V_7^3$?
* What is the number of elements in $V_{23}^0$,$V_{23}^1$,$V_{23}^2$ and $V_{23}^3$?

Now, compute the encodings of the following strings:

In [27]:
v0 = [1 0 1 1 0 1 1]'; # with the Hamming parity check matrix, as a column vector
v1 = [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0]'; # with the Golay parity check matrix

# Your code goes here



<a id=Dec></a>
<h2 style="color:#00A6D6;">Decoder</h2>


Now, let us take a look at the decoder. The protocol known as syndrome decoding works as follows: 
* Alice computes the syndrome $C_A = H \cdot X_A^T$. 
* Bob computes the syndrome $C_B=H\cdot X_B^T$ of his string $X_B$.
* The sum of $C_A$ and $C_B$ is fed into an estimator module that will output an estimate of the error string $\hat S$.
* The decoder outputs $\hat{X}_A= X_B+\hat S$

<img src="source/decoder.png">

We gave two parity check matrices in Exercise 1: $H_{\textrm{Hamming}}$ and $H_{\textrm{Golay}}$. They induce a unique encoder. However, there are many possible decoders that we could associate with them. If we look at the estimator module, it takes a syndrome and outputs an error estimate. We can choose the map between syndrome and outputs of the estimator module in many different ways. 

We are going to implement the following estimator map for the Hamming code:
* The zero syndrome gets mapped to the zero error string
* For each vector $v\in V^1_7$ compute the syndrome $s=H_{\textrm{Hamming}}\cdot v$ and assign the syndrome $s$ the error string $v$.

We have implemented this functionality as a Julia dictionary.

In [29]:
dict_Hamming = Dict()
z = 0*eVecInt(7,1)
s = 0*eVecInt(3,1)
dict_Hamming[s] = z
for i=1:7
    vi = eVecInt(7,i)
    s = getSyndrome(H_Hamming,vi)
    dict_Hamming[s] = vi
end

Now, if we want to know the output of the estimator for some concrete syndrome we would use the dictionary as follows:

In [30]:
s = [0 1 1]'
dict_Hamming[s]

7×1 Matrix{Int64}:
 0
 0
 0
 0
 0
 1
 0

<h2 style="color:#00A6D6;">Putting things together for the Hamming code </h2>

Let's have a look at how Alice and Bob would proceed in the case of the Hamming code. How many errors - i.e. how many flipped bits - can this procedure correct?

In [35]:
# First, let's give a value to Alice's string xA
xA = [0 0 1 0 0 1 1]';
print("Alice has string xA=",xA,"\n")

# Now Alice computes the syndrome cA and sends it to Bob
cA = getSyndrome(H_Hamming, xA);
print("Alice computes syndrome cA=",cA,"\n")

# Let's suppose that Bob actually received a noise version of the string, given by a specific noise pattern
noiseS = [0 0 0 0 1 0 0]';
print("Actual noise string S=",noiseS,"\n")
xB = xA + noiseS .% 2
print("Bob receives noisy string xB=",xB,"\n")

# Bob computes the syndrome of his noisy string
cB = getSyndrome(H_Hamming, xB)
print("Bob computes the syndrome cB=",cB,"\n")

# Let's add up the two syndromes
cS = (cA + cB) .% 2

# An estimate of the 
estimatedS = dict_Hamming[cS]
print("Estimated noise string hatS=",estimatedS, "\n")

# The corrected string that Bob outputs
tildeXB = (xB+estimatedS) .% 2
print("Bob's output string tildeXB=",tildeXB,"\n\n")

# Let's check whether that is equal to Alice's original input
print("Bob's output equals Alice'input: ", (tildeXB == xA))

Alice has string xA=[0; 0; 1; 0; 0; 1; 1;;]
Alice computes syndrome cA=[0; 1; 0;;]
Actual noise string S=[0; 0; 0; 0; 1; 0; 0;;]
Bob receives noisy string xB=[0; 0; 1; 0; 1; 1; 1;;]
Bob computes the syndrome cB=[1; 1; 1;;]
Estimated noise string hatS=[0; 0; 0; 0; 1; 0; 0;;]
Bob's output string tildeXB=[0; 0; 1; 0; 0; 1; 1;;]

Bob's output equals Alice'input: true

<h3 style="color:#00A6D6;"> Exercise 2</h3>

How many errors can the procedure above using the Hamming code tolerate? Investigate how many bits can be flipped above by the noise vector, and still lead to a correct decoding by Bob.

In [33]:
# Adapt noise in the code above, or copy paste it here to play around.

<h2 style="color:#00A6D6;">Putting things together for the Golay code </h2>

In the following exercises, let's investigate how Alice and Bob might use the Golay code in a similar fashion.

Now you can implement the following estimator map for the Golay code:

* The zero syndrome gets mapped to the zero error string
* For each vector $v\in V^1_{23}$ compute the syndrome $s=H_{\textrm{Golay}}\cdot v$ and assign the syndrome $s$ the error string $v$.
* For each vector $v\in V^2_{23}$ compute the syndrome $s=H_{\textrm{Golay}}\cdot v$ and assign the syndrome $s$ the error string $v$.
* For each vector $v\in V^3_{23}$ compute the syndrome $s=H_{\textrm{Golay}}\cdot v$ and assign the syndrome $s$ the error string $v$.

In [32]:
# Your code codes here. You may wish to start by extending the code given for the Hamming example above.
dict_Golay = Dict()

Dict{Any, Any}()

<h3 style="color:#00A6D6;"> Exercise 3</h3>

Compute the output of the estimator module of the Golay code with input error syndrome:

In [9]:
s0 = [1 1 1 0 0 0 0 1 1 1 0]
s1 = [1 0 1 0 0 1 1 1 0 1 0]

1×11 Matrix{Int64}:
 1  0  1  0  0  1  1  1  0  1  0

<h3 style="color:#00A6D6;"> Exercise 4</h3>

Now we can complete the decoder. Compute the output of the decoder for the Golay code with the following inputs.

In [10]:
# 1) 

Ca1 = [0 1 0 1 0 0 0 1 1 0 1]
Xb1 = [0 1 1 0 1 1 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0]

# 2)

Ca2 = [0 0 1 0 1 1 0 1 1 1 1]
Xb2 = [0 1 0 1 1 1 0 1 1 0 0 0 1 1 1 0 0 0 1 0 1 1 0]

1×23 Matrix{Int64}:
 0  1  0  1  1  1  0  1  1  0  0  0  1  1  1  0  0  0  1  0  1  1  0

In [11]:
function decoder(Ca,Xb,estimatorDict,H)
    # Here goes your code, you may wish to adapt the code given for the Hamming example abvove.
end

decoder (generic function with 1 method)


<h3 style="color:#00A6D6;"> Exercise 5</h3>

Given the following $X_A$ and $X_B$, what are the values of all the intermediate strings and what is the final decoder output?

In [12]:
Xa3 = [0 1 1 0 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1 1 0]
Xb3 = [0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 0 1 1 0]

1×23 Matrix{Int64}:
 0  1  1  0  1  0  0  1  1  0  0  1  1  0  0  1  0  0  1  0  1  1  0